In [ ]:
! pip install pandas
! pip install boto3
! pip install watchtower
! pip install s3fs==0.4.2
! pip install pyathena
! pip install matplotlib
! pip install scipy
! pip install ipywidgets
! pip install scikit-learn

! conda install -c conda-forge --yes implicit 

! pip install --upgrade jupyter_client # useful to make ipywidgets work properly when fitting data with implicit

# pip install git+https://gitlab.com/cloena/cloena-aws-tools.git

In [120]:
import pandas as pd

import s3fs

import pickle

import implicit

from aws_tools import athena_tools, s3_tools

import matplotlib

import re

import scipy

from ipywidgets import FloatProgress

import numpy as np

from sklearn.preprocessing import MinMaxScaler

In [121]:
_v = '_v0.2'

In [122]:
data = pd.read_csv('s3://bt-data-science-playground/bt-tv-recommendation-system/model_objects/historicalpurchasesrentals'+_v+'.csv')

In [123]:
data.head()

,ID_USER,TYPE_ASSET,TYPE_ENTITLEMENT,PRICE,PURCHASE_DATE,TITLE,TYPE,END_DATE,ID_EDITORIAL,GENRE,RATING
0,BBEU05449846,Film,TVOD,4.49,2020-05-27 00:03:22,Bad Boys for Life,film,2020-09-12 22:59:00,BBJ2550316A,Action,15
1,BBEU37034914,Film,TVOD,4.49,2020-05-27 08:52:36,Bad Boys for Life,film,2020-09-12 22:59:00,BBJ2550316A,Action,15
2,BBEU34520852,Film,TVOD,4.49,2020-05-27 09:27:01,Bad Boys for Life,film,2020-09-12 22:59:00,BBJ2550316A,Action,15
3,BBEU36931493,Film,TVOD,4.49,2020-05-27 13:15:53,Bad Boys for Life,film,2020-09-12 22:59:00,BBJ2550316A,Action,15
4,BBEU08836976,Film,TVOD,4.49,2020-05-27 13:07:31,Bad Boys for Life,film,2020-09-12 22:59:00,BBJ2550316A,Action,15


In [124]:
DROP_MUSIC = True
if DROP_MUSIC:
    data_implicit = data[data['TYPE_ASSET'] != 'Music']
else:
    data_implicit = data.copy()

In [125]:
# Calculate sparsity 
n_users = data_implicit['ID_USER'].unique().shape[0]
n_items = data_implicit['ID_EDITORIAL'].unique().shape[0]

print('Number of users: {}'.format(n_users))
print('Number of items: {}'.format(n_items))
print('Sparsity: {:4.3f}%'.format(float(data_implicit.shape[0]) / float(n_users*n_items) * 100))

Number of users: 353051
Number of items: 5728
Sparsity: 0.123%


In [126]:
data_implicit['eventStrength'] = 1

data_implicit['TITLE'] = data_implicit['TITLE'].astype("category")
data_implicit['ID_USER'] = data_implicit['ID_USER'].astype("category")
data_implicit['ID_EDITORIAL'] = data_implicit['ID_EDITORIAL'].astype("category")
data_implicit['ID_USER_simple'] = data_implicit['ID_USER'].cat.codes
data_implicit['ID_EDITORIAL_simple'] = data_implicit['ID_EDITORIAL'].cat.codes

data_implicit.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

,ID_USER,TYPE_ASSET,TYPE_ENTITLEMENT,PRICE,PURCHASE_DATE,TITLE,TYPE,END_DATE,ID_EDITORIAL,GENRE,RATING,eventStrength,ID_USER_simple,ID_EDITORIAL_simple
0,BBEU05449846,Film,TVOD,4.49,2020-05-27 00:03:22,Bad Boys for Life,film,2020-09-12 22:59:00,BBJ2550316A,Action,15,1,42776,2693
1,BBEU37034914,Film,TVOD,4.49,2020-05-27 08:52:36,Bad Boys for Life,film,2020-09-12 22:59:00,BBJ2550316A,Action,15,1,351262,2693
2,BBEU34520852,Film,TVOD,4.49,2020-05-27 09:27:01,Bad Boys for Life,film,2020-09-12 22:59:00,BBJ2550316A,Action,15,1,318888,2693
3,BBEU36931493,Film,TVOD,4.49,2020-05-27 13:15:53,Bad Boys for Life,film,2020-09-12 22:59:00,BBJ2550316A,Action,15,1,350470,2693
4,BBEU08836976,Film,TVOD,4.49,2020-05-27 13:07:31,Bad Boys for Life,film,2020-09-12 22:59:00,BBJ2550316A,Action,15,1,70679,2693


In [127]:
sparse_content_person = scipy.sparse.csr_matrix(
    (data_implicit['eventStrength'].astype(float), (data_implicit['ID_EDITORIAL_simple'], data_implicit['ID_USER_simple']))
)
sparse_person_content = scipy.sparse.csr_matrix(
    (data_implicit['eventStrength'].astype(float), (data_implicit['ID_USER_simple'], data_implicit['ID_EDITORIAL_simple']))
)

In [128]:
%%time

model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.1, iterations=50)
alpha = 15
data_tofit = (sparse_content_person * alpha).astype('double')
model.fit(data_tofit)


CPU times: user 3min 23s, sys: 647 ms, total: 3min 24s
Wall time: 13.6 s


In [129]:
# Save model object and data
data_implicit.to_csv('s3://bt-data-science-playground/bt-tv-recommendation-system/model_objects/modeldata'+_v+'.csv', index=False)
#
fs = s3fs.S3FileSystem(anon=False) 
with fs.open('s3://bt-data-science-playground/bt-tv-recommendation-system/model_objects/model'+_v+'.pkl', 'wb') as f:
        pickle.dump(model, f)

In [130]:
content_id = 1090
n_similar = 10

person_vecs = model.user_factors
content_vecs = model.item_factors

content_norms = np.sqrt((content_vecs * content_vecs).sum(axis=1)) # i.e. calculating abs. value of the vector of each item

scores = content_vecs.dot(content_vecs[content_id]) / content_norms # i.e. calculating cosine similarity, (A.B) / (|A| x |B|)
top_idx = np.argpartition(scores, -n_similar)[-n_similar:]
similar = sorted(zip(top_idx, scores[top_idx] / content_norms[content_id]), key=lambda x: -x[1])

for content in similar:
    idx, score = content
    print(data_implicit['TITLE'].loc[data_implicit['ID_EDITORIAL_simple'] == idx].iloc[0])

Wonder Woman
Justice League
Guardians of the Galaxy Vol. 2 [Bonus Edition]
Doctor Strange
Ant-Man
Guardians of the Galaxy
Batman V Superman: Dawn of Justice
Batman V Superman: Dawn of Justice [Ultimate Ed.]
Pacific Rim
Captain America: Civil War [Bonus Edition]


In [131]:
data_implicit[data_implicit['TITLE'].str.lower().str.contains("wonder woman")].head()

,ID_USER,TYPE_ASSET,TYPE_ENTITLEMENT,PRICE,PURCHASE_DATE,TITLE,TYPE,END_DATE,ID_EDITORIAL,GENRE,RATING,eventStrength,ID_USER_simple,ID_EDITORIAL_simple
949803,BBEU32064227,Film,TVOD,3.5,2019-09-04 11:17:24,Wonder Woman,film,2022-01-02 23:59:00,BBJ1442961A,Action,12,1,296447,1090
949804,BBEU29459636,Film,TVOD,3.5,2020-06-17 11:44:38,Wonder Woman,film,2022-01-02 23:59:00,BBJ1442961A,Action,12,1,268246,1090
949805,BBEU30178201,Film,TVOD,3.5,2019-12-08 16:29:07,Wonder Woman,film,2022-01-02 23:59:00,BBJ1442961A,Action,12,1,276581,1090
949806,BBEU19842331,Film,TVOD,3.5,2020-04-20 15:00:26,Wonder Woman,film,2022-01-02 23:59:00,BBJ1442961A,Action,12,1,141739,1090
949807,BBEU27589691,Film,TVOD,3.5,2019-08-14 20:18:23,Wonder Woman,film,2022-01-02 23:59:00,BBJ1442961A,Action,12,1,234140,1090


In [132]:
# Get the trained person and content vectors. We convert them to csr matrices
person_vecs = scipy.sparse.csr_matrix(model.user_factors)
content_vecs = scipy.sparse.csr_matrix(model.item_factors)

# Create recommendations for person with a given id 
person_id = 5

# Number of content items to get
num_contents=10

# Get the interactions scores from the sparse person content matrix
person_interactions = sparse_person_content[person_id,:].toarray()
# Add 1 to everything, so that articles with no interaction yet become equal to 1
person_interactions = person_interactions.reshape(-1) + 1
# Make articles already interacted zero
person_interactions[person_interactions > 1] = 0
# Get dot product of person vector and all content vectors
rec_vector = person_vecs[person_id,:].dot(content_vecs.T).toarray()

# Scale this recommendation vector between 0 and 1
min_max = MinMaxScaler()
rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
# Content already interacted have their recommendation multiplied by zero
recommend_vector = person_interactions * rec_vector_scaled
# Sort the indices of the content into order of best recommendations
content_idx = np.argsort(recommend_vector)[::-1][:num_contents]

# Start empty list to store titles and scores
titles = []
scores = []

for idx in content_idx:
    # Append titles and scores to the list
    titles.append(data_implicit['TITLE'].loc[data_implicit['ID_EDITORIAL_simple'] == idx].iloc[0])
    scores.append(recommend_vector[idx])

recommendations = pd.DataFrame({'title': titles, 'score': scores, 'ids' : content_idx})

print(recommendations)

                                               title     score   ids
0        Fantastic Beasts: The Crimes of Grindelwald  0.920562  2098
1                                 The Shape of Water  0.806978  1449
2                                            The Nun  0.792868  1959
3                                      Black Panther  0.786240  1532
4                                                 Us  0.778428  2265
5                                     A Star Is Born  0.758470  1997
6  The Chronicles of Narnia: The Lion, The Witch ...  0.753857  3107
7                                       Pet Sematary  0.744468  2335
8                            Jumanji: The Next Level  0.730752  2667
9                     Jurassic World: Fallen Kingdom  0.729755  1706


In [133]:
data_implicit[data_implicit['ID_USER_simple']==5].head(10)

,ID_USER,TYPE_ASSET,TYPE_ENTITLEMENT,PRICE,PURCHASE_DATE,TITLE,TYPE,END_DATE,ID_EDITORIAL,GENRE,RATING,eventStrength,ID_USER_simple,ID_EDITORIAL_simple
5288,BBEU00021416,Film,TVOD,4.49,2020-06-13 20:09:14,1917,film,2020-11-19 23:59:00,BBJ2526048A,Drama,15,1,5,2655
40668,BBEU00021416,Film,TVOD,4.49,2020-05-23 19:43:00,Birds Of Prey And the Fantabulous Emancipation...,film,2020-10-17 22:59:00,BBJ2569482A,Action,15,1,5,2733
64410,BBEU00021416,Film,EVOD,9.99,2020-06-06 18:07:13,The Lighthouse,film,2025-12-31 23:59:00,BBJ2536210A,Horror,15,1,5,2675
77014,BBEU00021416,Film,TVOD,4.49,2020-05-30 19:52:10,Little Women,film,2020-08-26 22:59:00,BBJ2417933A,Drama,u,1,5,2610
118110,BBEU00021416,Film,TVOD,3.50,2020-01-17 11:30:50,Descendants 3,film,2022-01-02 23:59:00,BBJ2341174A,Family,pg,1,5,2446
127247,BBEU00021416,Film,EVOD,5.49,2018-08-17 16:31:41,Despicable Me 3,film,2025-12-31 23:59:00,BBJ1453488A,Animation,u,1,5,1134
183749,BBEU00021416,Film,EVOD,9.99,2018-11-29 13:39:20,Tim Burton's The Nightmare Before Christmas,film,2025-12-31 23:59:00,BBJ1125474HVOD,Animation,pg,1,5,434
197297,BBEU00021416,Film,EVOD,15.99,2020-01-01 16:36:46,IT Chapter Two,film,2025-12-31 23:59:00,BBJ2398669A,Horror,15,1,5,2567
204200,BBEU00021416,Film,TVOD,4.00,2018-10-17 18:30:39,Harry Potter and the Half-Blood Prince,film,2022-01-02 23:59:00,BBJ956284HVOD,Fantasy,12,1,5,3923
235834,BBEU00021416,Film,TVOD,4.49,2020-03-14 20:52:52,Stephen King's Doctor Sleep,film,2020-07-10 22:59:00,BBJ2526229A,Horror,15,1,5,2660
